#  Twiiter Sentiment analysis on a presidential aspirant(Peter Obi) using NLP

# Content

# Introduction

This is a Data Analytics Project focused on Peter Obi tweets (presidental aspirant in Nigeria) using Natural Language Processing (NLP) Techniques. 
Every 4 year in Nigeria there is a change of govenmnet or extension of existing one, where every polictical party present their candidates, Peter Obi as one of the candidtes who seem to be widely talked about on twitter after he declared his ambition on one of the political parties. 

Sncrape was used to mine tweets about Peter Obi from March 2022 shortly before he declared his ambition till August 2022, the total tweets mined was 387,109k. This Notebook makes use of several Python libraries like Pandas (for Data Manipulation/Cleaning), Sncrape (for Tweets Mining), NLTK (Natural Language Toolkit libray to handle text data), TextBlob (for Sentiment Analysis), MatPlotlib, Plotly & WordCloud (for Data Visualization), Emot & emoticin (for Emojis identification/for expressed emotion emoji).


# Data mining and concatenation

Sncrape as a minning tool is as easy as bitting a pie, after trying a couple of tools including tweepy which requires regorous process for getting a substantial amount of data from twitter.
Snscrape is a scraper for social networking services (SNS), including facebook, Instagram etc. With it we were able to mine tweets containing 'peter obi' as our key word, data for six months was scrapped, i.e from March 4th 2022 till August 4th 2022, the data contains datatime, username, follower counts, like count, and more other variables.

# Import Libraries

Import all neccesary libraries to be used on this project. It is a convention way of importing the dependencies needed to be used.

In [ ]:
# import tweepy # for tweet mining
import pandas as pd
import numpy as np
import csv # to read and write csv files
import re # In-built regular expressions library
import string # Inbuilt sting library
import glob # to retrieve files/pathnames matching a specified pattern. 
import random # generating random numbers
import requests # to send HTTP requests
import os
import imageio.v2 as imageio
from PIL import Image # for opening, manipulating, and saving many different image file f
import matplotlib.pyplot as plt # for plotting

from nltk.corpus import stopwords # get stopwords from NLTK library
from nltk.tokenize import word_tokenize # to create word tokens
from nltk.stem.porter import * # (I played around with Stemmer and decided to use Lemmatizer instead)
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer # to reduce words to orginal form
from nltk import pos_tag # For Parts of Speech tagging
from nltk.corpus import words # Get all words in english language

#Text Sentiments
#TextBlob - Python library for processing textual data
import textblob
from textblob import TextBlob 

#Visualization tools
import plotly.express as px # To make express plots in Plotly
import chart_studio.tools as cst # For exporting to Chart studio
import chart_studio.plotly as py # for exporting Plotly visualizations to Chart Studio
import plotly.offline as pyo # Set notebook mode to work in offline
pyo.init_notebook_mode()
import plotly.io as pio # Plotly renderer
import plotly.graph_objects as go # For plotting plotly graph objects
from plotly.subplots import make_subplots

# WordCloud - Python linrary for creating image wordclouds
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

import chart_studio
username='olujoe'
api_key='your_api_key'
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)


In [ ]:
#load data
tweets_df = pd.read_csv('peter_obi_data.csv')

In [ ]:
# check for the data shape and view the dataframe
print('shape:', tweets_df.shape) 
tweets_df.head(5)

# Data Cleaning

through data cleaning proccess we are able to check for nan and none values in the dataframe, we wrw able to modify the data to ensure it is free of irrelevances and incorrect information. Date time was created to for an insight into the daily data trend. 

In [ ]:
 # Remove the unnamed column
tweets_df.drop(['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
#check for missing values, no missing value, as our data is free of nan values
tweets_df.isnull().sum()

In [ ]:
#Statitiscal insight into the data
tweets_df.describe()

In [ ]:
# info about our data
tweets_df.info()

In [ ]:
# convert timestamp column to datetime type
tweets_df['Datetime'] = pd.to_datetime(tweets_df['Datetime'])

In [ ]:
# extract date and time to create new columns 
tweets_df['Date'] = pd.to_datetime(tweets_df['Datetime']).dt.date
tweets_df['Time'] = pd.to_datetime(tweets_df['Datetime']).dt.time

In [ ]:
tweets_df['Date'] = pd.to_datetime(tweets_df['Date'])

In [ ]:
#check for the newly added columns
tweets_df.info()

# Handling text data

Text data i.e individual tweet contains charaters(emojis, numbers, #tags, unfamiliar words and stopwords) that can serve as noise in our data, which make it important for our data to be cleaned and properly treated. Some of the above libraries like NLTK helped us to manipulate our string data by changing the texts data into lower case, remove stopwaords, create additional columns for insight, remove emojis and lemmetize our data.

In [ ]:
#funtion for getting unique hashtags in the dataframe
def getHashtags(tweet):
    tweet = tweet.lower()  #has to be in place
    tweet = re.findall(r'\#\w+',tweet) # Remove hastags with REGEX
    return " ".join(tweet)

In [ ]:
#create #tag relative to each tweet.
tweets_df['Hashtags'] = tweets_df['Tweets'].apply(getHashtags)

In [ ]:
#create a list for other aspirants and aliases

other_aspirants = ['tinubu','asiwaju','bat','agbado','emi lokan', 'atiku', 'bobo chicago', 'jagaban', 'city boy',
                      'wike', 'kwankwaso', 'osinbajo', 'wike','saraki',
                      ]

In [ ]:
#function for generating other aspirants mentioned in peter_obi tweets

def get_other_aspirants(tweet):
    tweet = tweet.lower() # Reduce tweet to lower case
    tweet_tokens = word_tokenize(tweet) # split each word in the tweet for parsing
    presidential_aspirants = [char for char in tweet_tokens if char in other_aspirants]
    return " ".join(presidential_aspirants)

In [ ]:
tweets_df['Aspirants_mentioned'] = tweets_df['Tweets'].apply(get_other_aspirants)

In [ ]:
##generate ratio column

tweets_df['Ratioed'] = np.where(tweets_df['Replies_count'] > tweets_df['Likes_count'], 'Yes', 'No')

In [ ]:
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

In [ ]:
#  remove stopwords or some very common word manually and punctuation/ apply lemmatizer

# Defining my NLTK stop words and my user-defined stop words
stop_words = list(stopwords.words('english'))
user_stop_words = ['2023', '2019', 'much','pvc','president', 'abeg','presidential', 'election', 'amp', 'next', 'cant', 'wont', 'hadnt','havent', 'hasnt', 
                   'isnt', 'shouldnt', 'couldnt', 'wasnt', 'werent','mustnt', '’', '...', '..', '.', '.....', '....', 
                   'been…','aht', 've', 'next',"i'm","i'll","we'll","they'll","you'll","she'll","he'll","'ll","n't",
                   "'s",'anyone','today', 'lmao', 'lwkm','lol','yesterday','day', 'already']

alphabets = list(string.ascii_lowercase)
stop_words = stop_words + user_stop_words + alphabets
#word_list = words.words()  # all words in English language
emojis = list(UNICODE_EMOJI.keys())  # full list of emojis

In [ ]:
# Function to remove stop words, punctuations, emojis and return words to their base form using Lemmatizer

def preprocessTweets(tweet):
    tweet = tweet.lower()  # changes all words to lower case
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags = re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#\w+|\d+', '', tweet)
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)  # convert string to tokens
    filtered_words = [w for w in tweet_tokens if w not in stop_words]
    filtered_words = [w for w in filtered_words if w not in emojis]
    
    # Remove punctuations
    unpunctuated_words = [w for w in filtered_words if w not in string.punctuation]
    lemmatizer = WordNetLemmatizer() # instatiate an object WordNetLemmatizer Class
    lemma_words = [lemmatizer.lemmatize(w) for w in unpunctuated_words]
    return " ".join(lemma_words)

In [ ]:
# Apply preProcessTweets function to the 'Tweet' column to generate a new column called 'Processed Tweets'.
# This returns all words used to describe the TV series except for words in the lists above
tweets_df['Cleaned_Tweets'] = tweets_df['Tweets'].apply(preprocessTweets)

In [ ]:
tweets_df.head(5)

# Data exploratory analysis

### Charting and Visualization

In this section plotly was chosen as the vissualisation tool becuase of it flexibily and reactive screen, indights were gotten from the variables and a wordcloud was generated with the twitter bird ping to display the most frequent words.

In [ ]:
# create a chart for counts of users with most tweets
top_user = tweets_df.Username.value_counts().nlargest(10).to_frame()
top_user['name'] = top_user.index
top_user['Count'] = top_user['Username']
px.bar(top_user, x = 'name', y = 'Count', title = 'COUNT OF USERS WITH MOST TWEETS', color_discrete_sequence = ['darkred'])

In [ ]:
# create a chart for counts of users with most likes
max_likes =tweets_df.loc[tweets_df['Likes_count'] >= 30000]
px.bar(max_likes, x = 'Username', y = 'Likes_count', title = 'USERS WITH THE MOST LIKES', color_discrete_sequence = ['green'])

In [ ]:
# create a chart for counts of users with most retweets
max_retweets =tweets_df.loc[tweets_df['Retweet_count'] >= 7000]
px.bar(max_retweets, x = 'Username', y = 'Retweet_count', title = 'USERS WITH THE MOST RETWEET',color_discrete_sequence = ['mediumvioletred'], color = 'Ratioed')

In [ ]:
# create a chart for counts of users with most engagement
tweets_df['Engagement'] = tweets_df['Likes_count'] + tweets_df['Retweet_count'] + tweets_df['Replies_count'] + tweets_df['Quotes_count']
#tweets_df.head(5)
max_engagement =tweets_df.loc[tweets_df['Engagement'] >= 30000]
px.bar(max_engagement, x = 'Username', y = 'Engagement', title = 'USERS WITH THE MOST ENGAGEMENT', color = 'Ratioed')

In [ ]:
# create a chart for counts of users with most engagement and ratio
ratio_df =tweets_df.loc[tweets_df['Ratioed'] == 'Yes']
#ratio_df.head(5)
max_ratio = ratio_df.loc[ratio_df['Engagement'] >= 2000]

px.bar(max_ratio, x = 'Username', y = 'Engagement', title = 'USERS WITH THE MOST RATIO', color_discrete_sequence = ['darkred'])

In [ ]:
#create a daytime column for day
tweets_df['Day'] = tweets_df['Date'].dt.day_name()
# most active days with engagement 
colors = ['blue','green','black','purple','red', 'brown','cyan']
fig = px.pie(tweets_df, names = 'Day', title = 'MOST ACTIVE DAYS', color_discrete_sequence = ['green'])

fig.update_traces(hoverinfo = 'label + percent',textfont_size = 20, 
                 textinfo = 'label + percent', pull = [0.1,0,0,0,0,0,0],
                 marker = dict(colors = colors))

In [ ]:
# Extract all tweets into one long string with each word separate with a "space"
tweets_long_string = tweets_df['Cleaned_Tweets'].tolist()
tweets_long_string = " ".join(tweets_long_string)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import imageio.v2 as imageio

# join tweets to a single string
words = ' '.join(tweets_df['Cleaned_Tweets'])

# remove URLs, RTs, and twitter handles
neww = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

# create a twitter-style mask for the wordcloud
twitter_mask = imageio.imread('kfc.png')


STOPWORDS = stop_words

# generate the wordcloud
wordcloud = WordCloud(
                    stopwords = STOPWORDS,
                    background_color='white',
                    width=1800, 
                    height=1400, 
                    mask=twitter_mask
                    ).generate(neww)

# set the figure size
plt.figure(figsize=(12,12))

# show the wordcloud
plt.imshow(wordcloud)
plt.axis('off');

# Sentiment analysis

Finally we built a rule based sentiment analysis function to classify the sentiment behind the tweets; 

Polarity – talks about how positive or negative the opinion is

Subjectivity – talks about how subjective the opinion is

TextBlob(cleaned_tweets).sentiment gives us the Polarity, Subjectivity values.

Polarity ranges from -1 to 1 (1 is more positive, 0 is neutral, -1 is more negative)

Subjectivity ranges from 0 to 1(0 being very objective and 1 being very subjective)



In [ ]:
#  function to calculate subjrctivity
def getSubjectivity(cleaned_tweets):
    return TextBlob(cleaned_tweets).sentiment.subjectivity
#function to calculate sentiment
def getPolarity(cleaned_tweets):
    return TextBlob(cleaned_tweets).sentiment.polarity

#function to analyse the cleaned tweets
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutra'
    else:
        return 'positive'

In [ ]:
# create a dataframe for our cleaned_text data
sentiment_analysis = pd.DataFrame(tweets_df[['Cleaned_Tweets']])

In [ ]:

sentiment_analysis['Subjectivity'] = sentiment_analysis['Cleaned_Tweets'].apply(getSubjectivity)
sentiment_analysis['Polarity'] = sentiment_analysis['Cleaned_Tweets'].apply(getPolarity)
sentiment_analysis['Analysis'] = sentiment_analysis['Polarity'].apply(analysis)
sentiment_analysis.head(10)

In [ ]:


colors = ['blue','green','black','purple','red', 'brown','cyan']
fig = px.pie(sentiment_analysis, names = 'Analysis', title = 'PIE CHART OF SENTIMENT', color_discrete_sequence = ['green'])

fig.update_traces(hoverinfo = 'label + percent',textfont_size = 20,
                 textinfo = 'label + percent', pull = [0.1,0,0,0,0,0,0],
                 marker = dict(colors = colors))